In [15]:
import pandas as pd
import numpy as np
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score, recall_score
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold

import matplotlib.pyplot as plt
import dill
import sys
import seaborn as sns
# import the pipeline file
sys.path.insert(0,'../')
from utils import split_data

In [8]:
X_train,y_train = split_data.get_train_data()
X_test,y_test = split_data.get_test_data()
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25832 entries, 0 to 25831
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Time         25832 non-null  object 
 1   CellName     25832 non-null  object 
 2   PRBUsageUL   25832 non-null  float64
 3   PRBUsageDL   25832 non-null  float64
 4   meanThr_DL   25832 non-null  float64
 5   meanThr_UL   25832 non-null  float64
 6   maxThr_DL    25832 non-null  float64
 7   maxThr_UL    25832 non-null  float64
 8   meanUE_DL    25832 non-null  float64
 9   meanUE_UL    25832 non-null  float64
 10  maxUE_DL     25773 non-null  float64
 11  maxUE_UL     25773 non-null  float64
 12  maxUE_UL+DL  25828 non-null  object 
dtypes: float64(10), object(3)
memory usage: 2.6+ MB


In [9]:
with open('../pipelines/PreprocessingPipeline2.0.pkl', 'rb') as f:
    pipeline,categories = dill.load(f)
print(pipeline)

Pipeline(steps=[('SetNull', NullEncoder()),
                ('TimeConversion', TimeConversation()),
                ('ObjectToFloat', ObjectToFloat()), ('Scaling', Scaling()),
                ('OneHotEncoding',
                 OneHotEncoding(categories=array(['10ALTE', '10BLTE', '10CLTE', '1ALTE', '1BLTE', '1CLTE', '2ALTE',
       '3ALTE', '3BLTE', '3CLTE', '4ALTE', '4BLTE', '4CLTE', '5ALTE',
       '5BLTE', '5CLTE', '6ALTE', '6BLTE', '6CLTE', '6ULTE', '6VLTE',
       '6WLTE', '7ALTE', '7BLTE', '7CLTE', '7ULTE', '7VLTE', '7WLTE',
       '8ALTE', '8BLTE', '8CLTE', '9ALTE', '9BLTE'], dtype=object)))])


In [19]:
# Define the pipeline
pipeline = Pipeline([
    ('preprocessing', pipeline),
    ('imputer', SimpleImputer(strategy='mean')),
    ('classifier', None)
])

In [20]:
# Define the hyperparameter grid for Random Forest
rf_param_grid = {
    'classifier': [RandomForestClassifier()],
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [None, 5, 10],
    'classifier__min_samples_split': [2, 5],
}

# Define the hyperparameter grid for XGBoost
xgb_param_grid = {
    'classifier': [XGBClassifier()],
    'classifier__learning_rate': [0.1, 0.01],
    'classifier__max_depth': [3, 5],
    'classifier__n_estimators': [50, 100, 200],
}


In [23]:
# Define the hyperparameter grid for both classifiers
param_grid = [rf_param_grid, xgb_param_grid]

# Define the cross-validation strategy
cv = 5

# Define the GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid = param_grid, cv=cv, n_jobs=-1, scoring='recall')

# Fit the GridSearchCV object to the data
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and score
print('Best hyperparameters:', grid_search.best_params_)
print('Best score:', grid_search.best_score_)

/Users/nikulnayi/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/nikulnayi/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/nikulnayi/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/nikulnayi/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/Users/nikulnayi/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py", line 1901, in recall_score
    _, r, _, _ = precision_recall_fscore_support(
  File "/Users/nikulnayi/opt/anaconda3/lib/python3.9/sit

/Users/nikulnayi/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/nikulnayi/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/nikulnayi/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/nikulnayi/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/Users/nikulnayi/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py", line 1901, in recall_score
    _, r, _, _ = precision_recall_fscore_support(
  File "/Users/nikulnayi/opt/anaconda3/lib/python3.9/sit

/Users/nikulnayi/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/nikulnayi/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/nikulnayi/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/nikulnayi/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/Users/nikulnayi/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py", line 1901, in recall_score
    _, r, _, _ = precision_recall_fscore_support(
  File "/Users/nikulnayi/opt/anaconda3/lib/python3.9/sit

/Users/nikulnayi/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/nikulnayi/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/nikulnayi/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/nikulnayi/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/Users/nikulnayi/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py", line 1901, in recall_score
    _, r, _, _ = precision_recall_fscore_support(
  File "/Users/nikulnayi/opt/anaconda3/lib/python3.9/sit

/Users/nikulnayi/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/nikulnayi/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/nikulnayi/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/nikulnayi/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/Users/nikulnayi/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py", line 1901, in recall_score
    _, r, _, _ = precision_recall_fscore_support(
  File "/Users/nikulnayi/opt/anaconda3/lib/python3.9/sit

/Users/nikulnayi/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/nikulnayi/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/nikulnayi/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/nikulnayi/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/Users/nikulnayi/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py", line 1901, in recall_score
    _, r, _, _ = precision_recall_fscore_support(
  File "/Users/nikulnayi/opt/anaconda3/lib/python3.9/sit

Best hyperparameters: {'classifier': RandomForestClassifier(n_estimators=50), 'classifier__max_depth': None, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 50}
Best score: nan


In [22]:
print("Test set score:", grid_search.score(X_test, y_test))

Test set score: 0.506363027461487
